# IMPORT MongoDB

In [1]:
import pymongo
import re
import string
from xlsxwriter import Workbook
from pymongo import MongoClient

# Connection
Client = pymongo.MongoClient('localhost', 27017)
# Dataabase Selection
Database = Client['KPMG']
# Database Collections
IscrizioniCollection = Database['Iscrizioni']
ImmatricolazioniCollection = Database['Immatricolazioni']
FuorisedeCollection = Database['Fuorisede']
ISTAT_Collection = Database['ISTAT']

print('Available database -> ', Client.list_database_names())
print('Available collections -> ', Database.list_collection_names())

Available database ->  ['KPMG', 'Yelp', 'admin', 'config', 'local']
Available collections ->  ['Immatricolazioni', 'Iscrizioni', 'ISTAT', 'Fuorisede']


# Iscrizioni per Anno Accademico

In [2]:
result = IscrizioniCollection.aggregate([
    {
        '$project': {
            'GruppoCOD': 0, 
            'GruppoNOME': 0
        }
    }, {
        '$group': {
            '_id': [
                '$AnnoA', '$SESSO'
            ], 
            'Iscritti': {
                '$sum': '$Isc'
            }
        }
    }, {
        '$sort': {
            '_id': 1
        }
    }
])

res = list(result)
IscrizioniTotali=[]
i=0
while i < len(res):
    IscrizioniTotali.append({'AnnoAccademico' : res[i]['_id'][0], 'Iscritti' : (res[i]['Iscritti']+res[i+1]['Iscritti']),
                             'F' : res[i]['Iscritti'], 'M' : res[i+1]['Iscritti']})
    i+=2

# Iscrizioni per Settore

In [3]:
result = IscrizioniCollection.aggregate([
    {
        '$project': {
            'GruppoCOD': 0,
            'SESSO': 0
        }
    }, {
        '$group': {
            '_id': [
                '$AnnoA', '$GruppoNOME'
            ], 
            'Iscritti': {
                '$sum': '$Isc'
            }
        }
    }, {
        '$sort': {
            '_id': 1
        }
    }
])

res = list(result)
IscrizioniPerSettore=[]
i=0
for element in res:
    IscrizioniPerSettore.append({'AnnoAccademico' : element['_id'][0], 'Settore' : element['_id'][1],
                                 'Iscritti' : element['Iscritti']})

# Immatricolazioni per Anno Accademico

In [4]:
result = ImmatricolazioniCollection.aggregate([
    {
        '$project': {
            'GruppoNOME': 0
        }
    }, {
        '$group': {
            '_id': [
                '$AnnoA', '$SESSO'
            ], 
            'Immatricolati': {
                '$sum': '$Imm'
            }
        }
    }, {
        '$sort': {
            '_id': 1
        }
    }
])

res = list(result)
ImmatricolazioniTotali=[]
i=0
while i < len(res):
    ImmatricolazioniTotali.append({'AnnoAccademico' : res[i]['_id'][0],
                                   'Immatricolati' : (res[i]['Immatricolati']+res[i+1]['Immatricolati']),
                                   'F' : res[i]['Immatricolati'], 'M' : res[i+1]['Immatricolati']})
    i+=2

# Immatricolazioni per Settore

In [5]:
result = ImmatricolazioniCollection.aggregate([
    {
        '$project': {
            'SESSO': 0
        }
    }, {
        '$group': {
            '_id': [
                '$AnnoA', '$GruppoNOME'
            ], 
            'Immatricolati': {
                '$sum': '$Imm'
            }
        }
    }, {
        '$sort': {
            '_id': 1
        }
    }
])

res = list(result)
ImmatricolazioniPerSettore=[]
i=0
for element in res:
    ImmatricolazioniPerSettore.append({'AnnoAccademico' : element['_id'][0], 'Settore' : element['_id'][1],
                                       'Immatricolati' : element['Immatricolati']})

 # Immatricolati Fuorisede

In [6]:
result = FuorisedeCollection.aggregate([
    {
        '$project': {
            'AteneoCOD': 0, 
            'SedeP': 0
        }
    }, {
        '$sort': {
            'AnnoA': 1, 
            'AteneoNOME': 1
        }
    }
])

res = list(result)

remove_punct = dict((ord(punct),None) for punct in string.punctuation)
FuorisedeTotali = []
FuorisedePerAteneo = []
AnnoCorrente = ''
AteneoCorrente = ''
for element in res:
    NomeAteneo = element['AteneoNOME']
    NomeAteneo = NomeAteneo.translate(remove_punct)
    NomeAteneo = NomeAteneo.upper()
    NomeAteneo = NomeAteneo.split()
    if ((NomeAteneo[0] != element['ResidenzaR']) and
        (NomeAteneo[0] != element['ResidenzaP']) and (NomeAteneo[-1] != 'TELEMATICA')):
        if AnnoCorrente != element['AnnoA']:
            AnnoCorrente = element['AnnoA']
            AteneoCorrente = element['AteneoNOME']
            FuorisedeTotali.append({'AnnoAccademico' : element['AnnoA'], 'Fuorisede' : element['Imm']})
            FuorisedePerAteneo.append({'AnnoAccademico' : element['AnnoA'],
                                      'Ateneo': element['AteneoNOME'], 'Fuorisede' : element['Imm']})
        else:
            FuorisedeTotali[-1]['Fuorisede'] += element['Imm']
            if AteneoCorrente != element['AteneoNOME']:
                AteneoCorrente = element['AteneoNOME']
                FuorisedePerAteneo.append({'AnnoAccademico' : element['AnnoA'],
                                      'Ateneo': element['AteneoNOME'], 'Fuorisede' : element['Imm']})
            else:
                FuorisedePerAteneo[-1]['Fuorisede'] += element['Imm']

FuorisedePerAteneo = sorted(FuorisedePerAteneo, key=lambda d: d['Ateneo'])

# Fuorisede per Area di Provenienza

In [7]:
Atenei = ['Bologna', 'Milano', 'Roma La Sapienza','Napoli Federico II']

FilteredData = []
for element in res:
    if element['AteneoNOME'] in Atenei:
        FilteredData.append(element)
        
RegioniNord=['Provincia autonoma di Bolzano', "VALLE D'AOSTA",  'LIGURIA', 'LOMBARDIA', 'PIEMONTE',
             'Provincia autonoma di TRENTO', 'VENETO', 'FRIULI VENEZIA GIULIA', 'EMILIA ROMAGNA']
RegioniCentro=['TOSCANA', 'UMBRIA', 'MARCHE', 'LAZIO']
RegioniSud=['ABRUZZO', 'MOLISE', 'CAMPANIA', 'PUGLIA', 'BASILICATA', 'CALABRIA', 'SICILIA', 'SARDEGNA']        

FuorisedePerProvenienza = []
AteneoCorrente = ''
for element in FilteredData:
    if element['AteneoNOME'] != AteneoCorrente :
        AteneoCorrente = element['AteneoNOME']
        FuorisedePerProvenienza.append({'AnnoAccademico' : element['AnnoA'], 'Ateneo' : element['AteneoNOME'],
                                        'Nord' : 0, 'Centro' : 0, 'Sud' : 0})
    if element['ResidenzaR'] in RegioniNord:
        FuorisedePerProvenienza[-1]['Nord'] += element['Imm']
    elif element['ResidenzaR'] in RegioniCentro:
        FuorisedePerProvenienza[-1]['Centro'] += element['Imm']
    else:
        FuorisedePerProvenienza[-1]['Sud'] += element['Imm']

FuorisedePerProvenienza = sorted(FuorisedePerProvenienza, key=lambda d: d['Ateneo']) 

# Output dei Risultati

In [8]:
def FoglioExcel(Nome, ColumnName, Data):
    ws=wb.add_worksheet(Nome)
    first_row=0
    for element in ColumnnName:
        col=ColumnnName.index(element)
        ws.write(first_row,col,element)

    row=1
    for element in Data: 
        for _key,_value in element.items():
            col=ColumnnName.index(_key)
            ws.write(row,col,_value)
        row+=1

In [9]:
wb=Workbook("Risultati.xlsx")

ColumnnName=["AnnoAccademico", "Iscritti", "F", "M"]
FoglioExcel("Iscrizioni per A.A.", ColumnnName, IscrizioniTotali)

ColumnnName=["AnnoAccademico", "Settore", "Iscritti"]
FoglioExcel("Iscrizioni per Settore", ColumnnName, IscrizioniPerSettore)

ColumnnName=["AnnoAccademico", "Immatricolati", "F", "M"]
FoglioExcel("Immatricolazioni per A.A.", ColumnnName, ImmatricolazioniTotali)

ColumnnName=["AnnoAccademico", "Settore", "Immatricolati"]
FoglioExcel("Immatricolazioni per Settore", ColumnnName, ImmatricolazioniPerSettore)

ColumnnName=["AnnoAccademico", "Fuorisede"]
FoglioExcel("Fuorisede per A.A.", ColumnnName, FuorisedeTotali)

ColumnnName=["AnnoAccademico", "Ateneo", "Fuorisede"]
FoglioExcel("Fuorisede per Ateneo", ColumnnName, FuorisedePerAteneo)

ColumnnName=["AnnoAccademico", "Ateneo", "Nord", "Centro", "Sud"]
FoglioExcel("Fuorisede Per Provenienza", ColumnnName, FuorisedePerProvenienza)

wb.close()

# Costruzione Dataset

### Definizione Regioni e Province

In [10]:
Regioni = ["Valle d'Aosta / Vallée d'Aoste","Piemonte", "Liguria", "Lombardia", "Trentino Alto Adige / Südtirol", "Veneto",
          "Friuli-Venezia Giulia", "Emilia-Romagna", "Toscana", "Marche", "Umbria", "Lazio",
          "Abruzzo", "Molise", "Campania", "Puglia", "Basilicata", "Calabria", "Sicilia", "Sardegna"]
    

Province = {"Valle d'Aosta / Vallée d'Aoste" : ["Aosta"], "Piemonte" : ["Alessandria", "Asti", "Biella", "Cuneo", "Novara", "Torino",
            "Verbano-Cusio-Ossola", "Vercelli"], "Liguria" : ["Genova", "Imperia", "La Spezia", "Savona"],
            "Lombardia" : ["Bergamo", "Brescia", "Como", "Cremona", "Lecco", "Lodi", "Mantova", "Milano", "Monza e della Brianza",
            "Pavia", "Sondrio", "Varese"], "Trentino Alto Adige / Südtirol" : ["Bolzano / Bozen", "Trento"],
            "Veneto" : ["Belluno", "Padova", "Rovigo", "Treviso", "Venezia", "Verona", "Vicenza"],
            "Friuli-Venezia Giulia" : ["Gorizia", "Pordenone", "Trieste", "Udine"],
            "Emilia-Romagna" : ["Bologna", "Ferrara", "Forlì-Cesena", "Modena", "Parma", "Piacenza", "Ravenna",
            "Reggio nell'Emilia", "Rimini"], "Toscana" : ["Arezzo", "Firenze", "Grosseto", "Livorno", "Lucca", "Massa-Carrara",
            "Pisa", "Pistoia", "Prato", "Siena"], "Marche": ["Ancona", "Ascoli Piceno", "Fermo", "Macerata", "Pesaro e Urbino"],
            "Umbria" : ["Perugia", "Terni"], "Lazio" : ["Frosinone", "Latina", "Rieti", "Roma", "Viterbo"],
            "Abruzzo" : ["Chieti", "L'Aquila", "Pescara", "Teramo"], "Molise" : ["Campobasso", "Isernia"],
            "Campania" : ["Avellino", "Benevento", "Caserta", "Napoli", "Salerno"],
            "Puglia" : ["Bari", "Barletta-Andria-Trani", "Brindisi", "Foggia", "Lecce", "Taranto"],
            "Basilicata" : ["Matera", "Potenza"], "Calabria" : ["Catanzaro", "Cosenza", "Crotone", "Reggio di Calabria",
            "Vibo Valentia"], "Sicilia" : ["Agrigento", "Caltanissetta", "Catania", "Enna", "Messina", "Palermo",
            "Ragusa", "Siracusa", "Trapani"], "Sardegna" : ["Cagliari", "Nuoro", "Oristano", "Sassari", "Sud Sardegna"]}

### Input e Elaborazione dei dati sulla Popolazione

In [11]:
result = ISTAT_Collection.aggregate([
    {
        '$project': {
            'Territorio': 1, 
            'TIME': 1, 
            'CITTADINANZA': 1, 
            'Value': 1, 
            'ITTER107': 1
        }
    }, {
        '$match': {
            'Territorio': {
                '$not': {
                    '$in': [
                        re.compile(r"Italia"), re.compile(r"Nord"), re.compile(r"Centro"), re.compile(r"Mezzogiorno"), re.compile(r"Sud"), re.compile(r"Isole")
                    ]
                }
            }, 
            'CITTADINANZA': {
                '$in': [
                    re.compile(r"TOTAL")
                ]
            }, 
            'ITTER107': {
                '$not': {
                    '$eq': 'ITC20'
                }
            }
        }
    }, {
        '$project': {
            'CITTADINANZA': 0, 
            'ITTER107': 0
        }
    }, {
        '$sort': {
            'Territorio': 1, 
            'TIME': 1
        }
    }
])

res = list(result)

In [12]:
PopolazioneRegioni = []
Time = ['2010', '2011', '2012', '2013','2014', '2015', '2016', '2017','2018','2019']

for Regione in Regioni:
    i=0
    Popolazione = []
    for element in res:
        if element['Territorio'] == Regione and element['TIME'] == Time[i]:
            Popolazione.append(element['Value'])
            i+=1
    PopolazioneRegioni.append({'Regione' : Regione, 'Popolazione' : Popolazione})

In [13]:
PopolazioneProvince = []
Time = ['2010', '2011', '2012', '2013','2014', '2015', '2016', '2017','2018','2019']

for Regione in Regioni:
    for Provincia in Province[Regione]:
        i=0
        Popolazione=[]
        for element in res:
            if element['Territorio'] == Provincia and element['TIME'] == Time[i]:
                Popolazione.append(element['Value'])
                i+=1
        PopolazioneProvince.append({'Provincia' : Provincia, 'Popolazione' : Popolazione})

for Provincia in PopolazioneProvince:
    if Provincia['Provincia'] == "Aosta":
        Provincia['Popolazione'] = next((item for item in PopolazioneRegioni if item["Regione"] == "Valle d'Aosta / Vallée d'Aoste"), None)['Popolazione']

### Input e Elaborazione dei dati sugli Atenei


In [14]:
result = FuorisedeCollection.aggregate([
    {
        '$project': {
            'AnnoA': 1, 
            'AteneoNOME': 1, 
            'Imm': 1
        }
    }, {
        '$group': {
            '_id': [
                '$AteneoNOME', '$AnnoA'
            ], 
            'Immatricolati': {
                '$sum': '$Imm'
            }
        }
    }, {
        '$sort': {
            '_id': 1
        }
    }
])

res2 = list(result)
i=1
while i!=0:
    i=0
    for element in res2:
        if "telematica" in element['_id'][0] or "Stranieri" in element['_id'][0]:
            res2.remove(element)
            i=1

In [15]:
i=1
while i!=0:
    i=0
    AteneoCorrente = ''
    count = 0
    ListaOccorrenze = []
    for element in res2:
        if AteneoCorrente == element['_id'][0]:
            count += 1
        else:
            ListaOccorrenze.append({'Ateneo' : AteneoCorrente, 'Count' : count })
            count = 1
            AteneoCorrente = element['_id'][0]
    ListaOccorrenze = sorted(ListaOccorrenze, key=lambda d: d['Count']) 
    for item in ListaOccorrenze:
        if item['Count'] != 0 and item['Count'] < ListaOccorrenze[-1]['Count']:
            for element in res2:
                if element['_id'][0] == item['Ateneo']:
                    res2.remove(element)
                    i=1
    

In [16]:
FuorisedePerAteneo_2 = FuorisedePerAteneo
i=1
while i!=0:
    i=0
    AteneoCorrente = ''
    count = 0
    ListaOccorrenze = []
    for element in FuorisedePerAteneo_2:
        if AteneoCorrente == element['Ateneo']:
            count += 1
        else:
            ListaOccorrenze.append({'Ateneo' : AteneoCorrente, 'Count' : count })
            count = 1
            AteneoCorrente = element['Ateneo']
    ListaOccorrenze = sorted(ListaOccorrenze, key=lambda d: d['Count']) 
    for item in ListaOccorrenze:
        if item['Count'] != 0 and item['Count'] < ListaOccorrenze[-1]['Count']:
            for element in res2:
                if element['_id'][0] == item['Ateneo']:
                    res2.remove(element)
                    i=1
            for element in FuorisedePerAteneo_2:
                if element['Ateneo'] == item['Ateneo']:
                    FuorisedePerAteneo_2.remove(element)
                    i=1
                    
i=1
while i!=0:
    i=0
    for element in res2:
        if element['_id'][0] in ["Roma Europea","Roma UNINT"]:
            res2.remove(element)
            i=1

In [17]:
i=1
while i!=0:
    i=0
    for element in FuorisedePerAteneo_2:
        if element['AnnoAccademico'] == '2020/2021':
            FuorisedePerAteneo_2.remove(element)
            i=1
        
    for element in res2:
        if element['_id'][1] == '2020/2021':
            res2.remove(element)
            i=1

### Definizione del Dataset

In [18]:
remove_punct = dict((ord(punct),None) for punct in string.punctuation)

def ContaAteneiProvincia(Provincia):
    AteneoCorrente = ''
    Count = 0
    for Ateneo in res2:
        Nome_Ateneo = Ateneo['_id'][0]
        Nome_Ateneo = Nome_Ateneo.translate(remove_punct)
        Nome_Ateneo = Nome_Ateneo.split()
        if Nome_Ateneo[0] == Provincia:
            if AteneoCorrente != Ateneo['_id'][0]:
                AteneoCorrente = Ateneo['_id'][0]
                Count += 1
    return Count

def ContaAteneiRegione(Regione):
    Count = 0
    for Provincia in Province[Regione]:
        Count += ContaAteneiProvincia(Provincia)
    i=0
    while i<len(res2):
        if res2[i]['_id'][0] == Regione:
            Count += 1
            i = len(res2)
        i+=1
    return Count

In [19]:
Dataset = []

j=-1
for element in res2:
    j=(j+1)%10
    if element['_id'][0] in Regioni:
        PopolazioneR = next((item for item in PopolazioneRegioni if item["Regione"] == element['_id'][0]), None)['Popolazione'][j]
        Fuorisede = next((item for item in FuorisedePerAteneo_2 if (item["Ateneo"] == element['_id'][0]) and (item["AnnoAccademico"] == element['_id'][1])), None)['Fuorisede']
        AteneiR = ContaAteneiRegione(element['_id'][0])
        Dataset.append({'AnnoAccademico': element['_id'][1], 'Ateneo' : element['_id'][0],
                        'PopolazioneP': PopolazioneR,
                        'AteneiP' : AteneiR, 'PopolazioneR': PopolazioneR,
                        'AteneiR' : AteneiR, 'Fuorisede' : Fuorisede,
                        'Immatricolati' : element['Immatricolati']})
    else:
        NomeAteneo = element['_id'][0]
        NomeAteneo = NomeAteneo.translate(remove_punct)
        NomeAteneo = NomeAteneo.split()
        for Regione in Regioni:
                if NomeAteneo[0] in Province[Regione]:
                    PopolazioneR = next((item for item in PopolazioneRegioni if item["Regione"] == Regione), None)['Popolazione'][j]
                    PopolazioneP = next((item for item in PopolazioneProvince if item["Provincia"] == NomeAteneo[0]), None)['Popolazione'][j]
                    Fuorisede = next((item for item in FuorisedePerAteneo_2 if (item["Ateneo"] == element['_id'][0]) and (item["AnnoAccademico"] == element['_id'][1])), None)['Fuorisede']
                    AteneiR = ContaAteneiRegione(Regione)
                    AteneiP = ContaAteneiProvincia(NomeAteneo[0])
                    Dataset.append({'AnnoAccademico': element['_id'][1], 'Ateneo' : element['_id'][0],
                            'PopolazioneP': PopolazioneP,
                            'AteneiP' : AteneiP, 'PopolazioneR': PopolazioneR,
                            'AteneiR' : AteneiR, 'Fuorisede' : Fuorisede,
                            'Immatricolati' : element['Immatricolati']})

### Esportazione del Dataset

In [20]:
wb=Workbook("RegressionDataset.xlsx")

ColumnnName=["AnnoAccademico", "Ateneo", "PopolazioneP", "AteneiP","PopolazioneR", "AteneiR","Fuorisede","Immatricolati"]
FoglioExcel("Dataset", ColumnnName, Dataset)

wb.close()